<a href="https://colab.research.google.com/github/QaziSaim/DeepLearning--DeepNeuralNetwork-/blob/main/Practice_Basic_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gensim

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.utils import pad_sequences
import pandas as pd

In [4]:
!unzip /content/twitter_training.csv.zip -d /content/extraction_directory/

Archive:  /content/twitter_training.csv.zip
replace /content/extraction_directory/twitter_training.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/extraction_directory/twitter_training.csv  


In [5]:
df = pd.read_csv('/content/extraction_directory/twitter_training.csv')

In [6]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [7]:
df.columns

Index(['2401', 'Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')

In [8]:
df['2401'].unique()

array([2401, 2402, 2403, ..., 9198, 9199, 9200])

In [9]:

"""
array(['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)', 'NBA2K', 'Dota2', 'PlayStation5(PS5)',
       'WorldOfCraft', 'CS-GO', 'Google', 'AssassinsCreed', 'ApexLegends',
       'LeagueOfLegends', 'Fortnite', 'Microsoft', 'Hearthstone',
       'Battlefield', 'PlayerUnknownsBattlegrounds(PUBG)', 'Verizon',
       'HomeDepot', 'FIFA', 'RedDeadRedemption(RDR)', 'CallOfDuty',
       'TomClancysRainbowSix', 'Facebook', 'GrandTheftAuto(GTA)',
       'MaddenNFL', 'johnson&johnson', 'Cyberpunk2077',
       'TomClancysGhostRecon', 'Nvidia'], dtype=object)

"""
df['Borderlands'].unique()

array(['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)', 'NBA2K', 'Dota2', 'PlayStation5(PS5)',
       'WorldOfCraft', 'CS-GO', 'Google', 'AssassinsCreed', 'ApexLegends',
       'LeagueOfLegends', 'Fortnite', 'Microsoft', 'Hearthstone',
       'Battlefield', 'PlayerUnknownsBattlegrounds(PUBG)', 'Verizon',
       'HomeDepot', 'FIFA', 'RedDeadRedemption(RDR)', 'CallOfDuty',
       'TomClancysRainbowSix', 'Facebook', 'GrandTheftAuto(GTA)',
       'MaddenNFL', 'johnson&johnson', 'Cyberpunk2077',
       'TomClancysGhostRecon', 'Nvidia'], dtype=object)

In [10]:
df.drop(columns=['2401', 'Borderlands'],inplace=True)

In [11]:
df.head()

,Positive,"im getting on borderlands and i will murder you all ,"
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [12]:
df.columns

Index(['Positive', 'im getting on borderlands and i will murder you all ,'], dtype='object')

In [13]:
df.Positive.unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [14]:
df.rename(columns={'Positive':'Target','im getting on borderlands and i will murder you all ,':'text'},inplace=True)

In [15]:
df.head()

,Target,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [16]:
df.Target.unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [17]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['Positive', 'Neutral', 'Negative', 'Irrelevant']])
df.Target = encoder.fit_transform(df[['Target']])

In [18]:
df.head()

,Target,text
0,0.0,I am coming to the borders and I will kill you...
1,0.0,im getting on borderlands and i will kill you ...
2,0.0,im coming on borderlands and i will murder you...
3,0.0,im getting on borderlands 2 and i will murder ...
4,0.0,im getting into borderlands and i can murder y...


In [19]:
df.Target.unique()

array([0., 1., 2., 3.])

In [20]:
df.head()

,Target,text
0,0.0,I am coming to the borders and I will kill you...
1,0.0,im getting on borderlands and i will kill you ...
2,0.0,im coming on borderlands and i will murder you...
3,0.0,im getting on borderlands 2 and i will murder ...
4,0.0,im getting into borderlands and i can murder y...


In [21]:
df.duplicated().sum()

4909

In [22]:
df.isnull().sum()

,0
Target,0
text,686


In [23]:
df.Target.value_counts()

,count
Target,
2.0,22542
0.0,20831
1.0,18318
3.0,12990


In [24]:
df.drop_duplicates(inplace=True)

In [25]:
df.dropna(inplace=True)

In [26]:
df.Target.value_counts()

,count
Target,
2.0,21237
0.0,19137
1.0,17110
3.0,12284


In [27]:
X = df.text

In [28]:
y = df.Target

In [29]:
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
text_lines = list()
for text in X:
  tokens = word_tokenize(text)
  tokens = [n.lower() for n in tokens]
  table  = str.maketrans('','',string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  words = [word for word in stripped if word.isalpha()]

  stop_words = set(stopwords.words('english'))
  words = [word for word in words if word not in stop_words]
  text_lines.append(words)

In [31]:
text_lines

[['coming', 'borders', 'kill'],
 ['im', 'getting', 'borderlands', 'kill'],
 ['im', 'coming', 'borderlands', 'murder'],
 ['im', 'getting', 'borderlands', 'murder'],
 ['im', 'getting', 'borderlands', 'murder'],
 ['spent',
  'hours',
  'making',
  'something',
  'fun',
  'nt',
  'know',
  'huge',
  'borderlands',
  'fan',
  'maya',
  'one',
  'favorite',
  'characters',
  'decided',
  'make',
  'wallpaper',
  'pc',
  'original',
  'image',
  'versus',
  'creation',
  'made',
  'enjoy'],
 ['spent',
  'couple',
  'hours',
  'something',
  'fun',
  'nt',
  'know',
  'huge',
  'borderlands',
  'fan',
  'maya',
  'one',
  'favorite',
  'characters',
  'decided',
  'make',
  'wallpaper',
  'pc',
  'original',
  'picture',
  'compared',
  'creation',
  'made',
  'fun',
  'pictwittercom'],
 ['spent',
  'hours',
  'something',
  'fun',
  'nt',
  'know',
  'huge',
  'borderlands',
  'fan',
  'maya',
  'one',
  'favorite',
  'characters'],
 ['spent',
  'hours',
  'making',
  'something',
  'fun',
  

In [32]:
len(text_lines)

69768

In [33]:
df.shape

(69768, 2)

In [34]:
import gensim

In [38]:
EMBEDDING_DIM = 100
model = gensim.models.Word2Vec(sentences=text_lines,vector_size=EMBEDDING_DIM,window=5,min_count=1)

words = list(model.wv.index_to_key)
print(f'Vocabulary size: {len(words)}')


Vocabulary size: 30558


In [47]:
model.wv.most_similar('university',topn=5)

[('craftsman', 0.9887738823890686),
 ('magnolia', 0.9884276390075684),
 ('fearrts', 0.9880506992340088),
 ('natural', 0.9879912734031677),
 ('feom', 0.987945556640625)]

In [61]:
model.wv.get_vector('cut')

array([-8.47280860e-01, -4.03909415e-01,  1.47591665e-01,  1.17910266e-01,
       -1.73575193e-01, -5.72943270e-01,  1.86147109e-01,  5.43679535e-01,
       -3.09719175e-01, -5.26360311e-02, -7.96521068e-01,  2.50944197e-01,
       -1.24988547e-02, -4.06984001e-01, -5.36388345e-02, -1.85074806e-01,
        1.10512592e-01,  1.93747625e-01, -3.40203166e-01, -5.08732200e-01,
       -2.25244779e-02, -8.58322624e-03,  2.17502058e-01,  3.70254013e-04,
        9.52805802e-02, -1.53335795e-01, -1.50131717e-01, -5.72724640e-01,
       -4.53134060e-01,  1.40104711e-01, -2.46044010e-01, -1.00207902e-01,
        4.25868988e-01, -1.57224700e-01,  2.66083866e-01,  4.35804516e-01,
       -2.05845162e-01, -6.20590091e-01,  3.54002155e-02, -1.90538451e-01,
        5.08700073e-01,  1.17996901e-01, -3.98605347e-01, -7.74038285e-02,
        7.17678189e-01, -1.03403039e-01,  1.00130975e-01, -4.31831211e-01,
        4.37735826e-01, -4.13857192e-01,  9.07946110e-01, -8.29963088e-01,
       -6.08550787e-01, -

In [43]:
print()

In [62]:
filename = 'twitter_comments_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)


In [63]:
model.wv.most_similar(positive=['happy','smile'],negative='sad')

[('reigns', 0.8560488224029541),
 ('hailey', 0.8382306694984436),
 ('irisnursophea', 0.8344045877456665),
 ('shikharthebun', 0.8342871069908142),
 ('thanks', 0.8329585194587708),
 ('raid', 0.8326830863952637),
 ('booking', 0.8302150964736938),
 ('voted', 0.8272694945335388),
 ('clocksydraws', 0.826509952545166),
 ('girft', 0.8260785341262817)]

In [64]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

[('rapids', 0.9345349669456482),
 ('maxoakdirect', 0.9329218864440918),
 ('thoughts', 0.9307766556739807),
 ('ins', 0.9304002523422241),
 ('investing', 0.9243392944335938),
 ('covers', 0.9233654737472534),
 ('maldraxxus', 0.9205113053321838),
 ('proposal', 0.9203864336013794),
 ('noticias', 0.9186923503875732),
 ('annnieanna', 0.9186356067657471)]

In [75]:
model.wv.similarity('men','happy')

0.3891927

In [77]:
model.wv.similar_by_vector(10,50)

[('comwatch', 0.834473192691803),
 ('gooo', 0.80033940076828),
 ('dammit', 0.7831889390945435),
 ('gah', 0.7823163866996765),
 ('disgrace', 0.7821529507637024),
 ('hahaha', 0.7812962532043457),
 ('exists', 0.7689480185508728),
 ('agerwalnidhhi', 0.7681468725204468),
 ('goooo', 0.7647524476051331),
 ('carrrrr', 0.7646674513816833),
 ('sucker', 0.7640823721885681),
 ('alive', 0.7600603103637695),
 ('century', 0.7598835229873657),
 ('imba', 0.7596852779388428),
 ('disney', 0.7588689923286438),
 ('educate', 0.7587639689445496),
 ('boi', 0.75872403383255),
 ('yeah', 0.757278323173523),
 ('fazesway', 0.755813717842102),
 ('pt', 0.7546807527542114),
 ('n', 0.7541388273239136),
 ('lmfaoooooo', 0.7536212801933289),
 ('guitar', 0.7533186674118042),
 ('boating', 0.7531319856643677),
 ('omg', 0.7520838975906372),
 ('kyoot', 0.7517712712287903),
 ('poqijk', 0.7495370507240295),
 ('goose', 0.7479863166809082),
 ('hilarious', 0.7479691505432129),
 ('exploding', 0.7478535175323486),
 ('yes', 0.7463197

In [79]:
embeddings_index = {}
f = open('/content/twitter_comments_embedding_word2vec.txt')
for line in f:
    print(line)
    values = line.split()
    word = values[0] # Since the word is at the 0th index
    coefs = np.asarray(values[1:]) # From the 1st position onwards the values are the embedding values
    embeddings_index[word] = coefs
f.close()


Streaming output truncated to the last 5000 lines.
withdra -0.012358195 -0.009205968 -0.0067456495 -0.003913709 0.00038053226 0.0031897307 0.0014461395 0.00509787 -0.00065717165 -0.00010168523 -0.014729679 0.0057519716 -0.011697884 -0.007914953 -0.004949574 0.00014607978 0.00052540406 0.0071885786 0.00635659 -0.0017435632 1.5923171e-05 -0.0002187554 0.0104360115 0.0067428057 -0.008795614 0.00091608235 0.0020557113 -0.0076209754 -0.009553993 -0.004238742 -0.0029942894 0.0027914492 -0.00067347486 -0.011629978 0.007958711 -0.0030110097 -0.0101379035 -0.015491321 0.007888887 -0.0076542073 0.009738054 0.010235231 0.00063199445 0.004714264 0.000556137 -0.0036385448 -0.007441441 -0.010928604 -0.001518412 0.00897684 0.0063305227 -0.0024086016 -0.013128099 -0.0038606063 0.010797232 -0.00047253232 -0.009219125 0.0040357304 -0.0114011485 0.0054838974 0.008453754 0.005984225 -0.0019804565 -0.0084921215 -0.007964997 0.002491351 -0.009239046 0.002690384 -0.0068448028 -0.0016987582 -0.014140055 0.012

In [81]:
!pip install keras_hub

In [83]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
VALIDATION_SPLIT = 0.2
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(text_lines)
sequences = tokenizer_obj.texts_to_sequences(text_lines) # Transforms text into a sequence of integers
